In [3]:
import os
import requests
import numpy as np
import duckdb
import polars as pl
from dotenv import load_dotenv


In [4]:
load_dotenv()

True

In [5]:
from typing import Dict, Optional, List, Any

In [6]:
from anthropic.types import Message, TextBlock, ToolUseBlock
from claudette.core import *

### Define Model 

In [7]:
model = models[-1]
model

'claude-3-haiku-20240307'

### Define Tools

In [8]:
sp = """
You are an expert at generating statements of work for government contracting. You create statements of work(SOW) that are
clear, concise, and adhere to best practices. Please use the provided examples to inform the style and best practices for your own
SOW's.

# STATEMENT OF WORK (SOW) EXAMPLES

EXAMPLE 1 - Project Title: Pinewoods RO Treatment Plant and Wellfield Phase II

A. Introduction/Background

Lee County Utilities (LCU) owns and operates the Pinewoods Water Treatment Plant and related
public water supply wellfield south of Corkscrew Road approximately one mile east of Interstate I-

75. Currently, the wellfield consists of wells withdrawing water from traditional sources, the
surficial and sandstone aquifers. Given the competition for fresh water resources and potential
wetland impacts LCU decided that further development of raw water from traditional sources in
this area was not feasible. Therefore, it was decided that further wellfield development would be
raw water from the Lower Hawthorne aquifer, an alternative source. Lee County has initiated the
construction of Pinewoods Lower Hawthorne Wells and RO Treatment Plant Phase I. In phase I
the storage tank and RO building were completed. The next phase of this project will consist of
completion of the 3.2 million gallons per day (MGD) Reverse Osmosis (RO) Water Treatment
Plant (WTP) and the related Lower Hawthorne Wellfield.

B. Objectives

The objective of this project is to assist in the completion of the Pinewoods Reverse Osmosis
Water Treatment Plant and Lower Hawthorne Wellfield Phase II to help meet the City's water
supply demands.

C. Scope of Work

Fiscal Year 2007 work will complete construction of a 3.2 mgd water treatment facility and four
Lower Hawthorne Floridan Aquifer Production wells. Work will include construction of RO
equipment, high service pumps, mechanical equipment, yard piping, wellfield piping, electrical
equipment, production wells, monitoring wells, raw water pipelines, and related appurtenances.

EXAMPLE 2 - Project Title: Floridan Aquifer Wells (2) at Dixie Wellfield

A Introduction/Background

The project titled “Dixie Brackish Water Supply / Treatment Facility” is composed of multiple
phases as follows:

- Phase 1 - Construct Two Floridan Aquifer Test Wells
- Phase 2 - Conceptual Plan for Brackish Water Supply and Treatment
- Phase 3 - RO Membrane Pilot Testing
- Phase 4 - Floridan Aquifer Production Well Design & Construction
- Phase 5 - Wellfield Conveyance Facilities Design & Construction
- Phase 6 - Dixie Reverse Osmosis Facilities Design & Construction

This project includes construction work associated with Phase 1, which includes the installation
and testing of two (2) 24-inch diameter by approximately 1,100-foot deep water supply wells to be
completed in the Floridan Aquifer within the City of Fort Lauderdale's Dixie Wellfield. These wells
will provide raw water to the Peele-Dixie Water Treatment Plant.

B Objectives

The objective of this project is to assist in the construction of a well field necessary to supply
Floridan Aquifer water to help meet the City's water supply demands.


C Scope of Work

Fiscal Year 2007 work will consist of construction and testing of two 24-inch diameter water
supply wells completed in the Floridan Aquifer. The wells are anticipated to able about 1,100 feet
in depth and will be located at the City of Fort Lauderdale's Dixie Wellfield. These wells will
provide raw water to the Peele-Dixie Water Treatment Plant.

EXAMPLE 3 - Project Title: Floridan Aquifer Wells (2) and Two (2) MGD RO Expansion

A Introduction/Background

The City of Hollywood WTP requires additional treated capacity to service its existing and
developing customer base. Additional raw water supply to be able to produce additional treated
capacity is required in the short term. This request includes construction of two (2) new Floridan
Aquifer groundwater wells classified as proposed under existing Consumptive Use Permit (CUP).
These wells have previously been identified in the Lower East Coast Water Supply
Questionnaire. In addition, a new 2 MGD Reverse Osmosis (RO) train is required to be built to
increase treated capacity added to the distribution system from the Floridan water supply.

Proposed Floridan Aquifer production wells requested under this application have already been
characterized in the existing CUP. This project converts proposed production wells into existing
test production wells, augmenting available supply from the Floridan Aquifer. Plans to revise the
CUP accordingly are underway.

Construction of new 2 MGD RO train has limited permitting requirements which will be obtained
during the Bid Preparation stage indicated on the attached schedule. Hollywood RO system has
been designed and built for modular expansion. RO train to be constructed will be replication of
existing 2 MGD train and will be fabricated by skid supplier according to pre-established design
specifications.

B Objectives

The objective of this project is to help meet the additional treated capacity required to service the
City's existing developing customer base by increasing the Floridan Aquifer raw water supply and
the Reverse Osmosis water treatment plant capacity.

C Scope of Work

This project entails the construction of two (2) Floridan Aquifer test production wells (16”
diameter, cased to 950') and installation of an additional 2 MGD skid-mounted RO train starting
from the pump discharge and terminating at the concentrate and permeate headers within the
skid.

The two (2) Floridan Alternative Water Supply wells (estimated at $920,000 per well, 16”
diameter, cased to 950') are contained in the existing Consumptive Use Permit. These wells
previously have been identified in the Lower East Coast Water Supply Questionnaire. Wells will
be constructed as “test” production wells in this phase to enable water quality analysis,
confirmation of hydro-geologic conditions, pump selection, and raw water pipeline design to be
performed for construction in subsequent phase.

The new two (2) MGD RO membrane train (approximately $619,600 cost) will be constructed
from existing design information to increase the proportion of Plant treated water originating from
the Floridan Aquifer Alternative Water Supply and allow utilization of new production wells also
contained in this request.


EXAMPLE 4 - Project Title: Reclaimed Water Booster Pump Station (North-South
Interconnect Pipes under Livingston Road)

A Introduction/Background

Collier County reused approximately 5 billion gallons of treated wastewater in 2004. The majority
of the water is used by commercial and residential users for irrigation. Collier County also uses
reclaimed water for irrigation at various parks and roadway medians. A small portion of the
reclaimed water is used for environmental mitigation. The reclaimed water system is made up of
over 50 miles of distribution piping, 130 million gallons of wet weather storage, two deep injection
wells, and six large pumping stations.

The County serves two reclaimed water service areas referred to as the South and North service
areas. The South service area is primarily served by treated effluent from the South County
Water Reclamation Facility (WRF) and the North service area is served by the North County
WRF. The main issue Collier County's reclaimed water program faces is wet weather storage.

The County has identified a need to construct an in-line booster pump station to add flexibility and
increase efficiency to their reclaimed water system. The reclaimed water pump station will
enhance the interconnections of the North and South reclaimed water service areas, allowing the
County to serve as many customers as possible and also maximizing the use of their reclaimed
water storage facilities, including the reclaimed water ASR system, located in the North service
area.

B Objectives

This project will allow storage of reclaimed water during the wet season, providing additional
reclaimed water during the peak demand. The project will consist of two phases: Phase 1 is to
connect the North and South reclaimed water service areas by microtunneling under the
Livingston Road; Phase 2 is to build a reclaimed water booster pumping station.

C Scope of Work

FY2007 work activities include Phase 1: the microtunneling of two 16-inch reclaimed water mains
across Livingston Road.

EXAMPLE 5 - Project Title: JEA 16.5 MGD Floridan Wellfield and RO Expansion Ph II

A Introduction/Background

The City of Port St. Lucie's has a water and waste water master plan. Sources of water for the
City in the near future are anticipated to be from the Floridan Aquifer. Major impediments to
increasing the use of alternative water supplies includes competition with are communities for
allocations from the same aquifer water source. This project will provide substantial
environmental benefits by replacing water typically supplied from the Surficial Aquifer sources
with brackish water from the Floridan Aquifer. It will also reduce competition for water supplies by
reducing the quantity of Surficial Aquifer source water needed by the City of Port St. Lucie.


B Objectives

To expand the JEA Reverse Osmosis Water Treatment Plant capacity from 6.0 MGD to 22.
MGD by increasing the numbers of Floridan wells and the number of reverse osmosis membrane
trains.

C Scope of Work

Project work includes construction of water mains for raw water wells to supply water from the
Floridan aquifer and expansion of the JEA Reverse Osmosis Water Treatment Plant from 6.
MGD to 22.5 MGD Phase II of this work is for the water mains for raw water wells F-13, F-14, and
F-15.


"""

In [9]:
text_input = """There has been an increase of traffic and wear and tear on main st. in charlottesville, va.
Can you create a statement of work for repaving it? There will need to be at least 3 applications of asphalt and sealant. 
We will also have to proceed in phases because we can only disrupt traffic for a week at a time. Let's do 3-5 phases."""

In [10]:
sow_chat = Chat(model=model, sp=sp)

In [11]:
sow_chat(text_input)

STATEMENT OF WORK (SOW)

Project Title: Repaving of Main Street, Charlottesville, VA

A. Introduction/Background
The City of Charlottesville has experienced an increase in traffic and wear and tear on Main Street, necessitating a comprehensive repaving project. The repaving will involve a multi-phase approach to minimize disruption to traffic and ensure the efficient completion of the work.

B. Objectives
The primary objectives of this project are to:
1. Restore the structural integrity and surface condition of Main Street through a multi-layered asphalt application and sealant treatment.
2. Implement the repaving work in a phased approach to maintain traffic flow and minimize disruptions to the local community.

C. Scope of Work
The scope of work for this project includes the following:

Phase 1:
1. Prepare the existing road surface by milling and removing the top layer of asphalt.
2. Apply the first layer of asphalt, with a minimum thickness of 2 inches.
3. Apply a sealant coating to the new asphalt surface.
4. Restore all affected areas, including curbs, gutters, and sidewalks, to their original condition.
5. Implement traffic control measures, such as lane closures and detours, to facilitate the work while maintaining access to local businesses and residences.

Phase 2:
1. Repeat the process outlined in Phase 1, including milling, applying a second layer of asphalt (minimum 2 inches), and applying a sealant coating.
2. Continue to implement traffic control measures to minimize disruptions to the local community.

Phase 3:
1. Repeat the process outlined in Phase 1 and 2, including milling, applying a third layer of asphalt (minimum 2 inches), and applying a sealant coating.
2. Finalize all restoration work, including curbs, gutters, and sidewalks.
3. Implement final traffic control measures to ensure the safe and efficient completion of the project.

D. Project Schedule
The project is expected to be completed within a 3-5 phase approach, with each phase lasting approximately one week. The specific schedule will be coordinated with the City of Charlottesville to minimize disruptions to local businesses and residents.

E. Deliverables
1. Repaved Main Street with a minimum of three layers of asphalt and sealant coating.
2. Restored curbs, gutters, and sidewalks to their original condition.
3. Maintained traffic flow and access to local businesses and residences throughout the project.

F. Project Management and Coordination
The Contractor shall work closely with the City of Charlottesville to ensure the successful completion of the project. Regular progress meetings and coordination with local stakeholders will be required to address any concerns or issues that may arise during the course of the work.

<details>

- id: msg_01Xj8YnWVzUxMnD6yoN4ZKoC
- content: [{'text': 'STATEMENT OF WORK (SOW)\n\nProject Title: Repaving of Main Street, Charlottesville, VA\n\nA. Introduction/Background\nThe City of Charlottesville has experienced an increase in traffic and wear and tear on Main Street, necessitating a comprehensive repaving project. The repaving will involve a multi-phase approach to minimize disruption to traffic and ensure the efficient completion of the work.\n\nB. Objectives\nThe primary objectives of this project are to:\n1. Restore the structural integrity and surface condition of Main Street through a multi-layered asphalt application and sealant treatment.\n2. Implement the repaving work in a phased approach to maintain traffic flow and minimize disruptions to the local community.\n\nC. Scope of Work\nThe scope of work for this project includes the following:\n\nPhase 1:\n1. Prepare the existing road surface by milling and removing the top layer of asphalt.\n2. Apply the first layer of asphalt, with a minimum thickness of 2 inches.\n3. Apply a sealant coating to the new asphalt surface.\n4. Restore all affected areas, including curbs, gutters, and sidewalks, to their original condition.\n5. Implement traffic control measures, such as lane closures and detours, to facilitate the work while maintaining access to local businesses and residences.\n\nPhase 2:\n1. Repeat the process outlined in Phase 1, including milling, applying a second layer of asphalt (minimum 2 inches), and applying a sealant coating.\n2. Continue to implement traffic control measures to minimize disruptions to the local community.\n\nPhase 3:\n1. Repeat the process outlined in Phase 1 and 2, including milling, applying a third layer of asphalt (minimum 2 inches), and applying a sealant coating.\n2. Finalize all restoration work, including curbs, gutters, and sidewalks.\n3. Implement final traffic control measures to ensure the safe and efficient completion of the project.\n\nD. Project Schedule\nThe project is expected to be completed within a 3-5 phase approach, with each phase lasting approximately one week. The specific schedule will be coordinated with the City of Charlottesville to minimize disruptions to local businesses and residents.\n\nE. Deliverables\n1. Repaved Main Street with a minimum of three layers of asphalt and sealant coating.\n2. Restored curbs, gutters, and sidewalks to their original condition.\n3. Maintained traffic flow and access to local businesses and residences throughout the project.\n\nF. Project Management and Coordination\nThe Contractor shall work closely with the City of Charlottesville to ensure the successful completion of the project. Regular progress meetings and coordination with local stakeholders will be required to address any concerns or issues that may arise during the course of the work.', 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 2424, 'output_tokens': 645}

</details>

In [12]:
sow_chat.use

In: 2424; Out: 645; Total: 3069

In [20]:
completion_tokens = sow_chat.use.model_dump()

In [21]:
haiku_cost = {'input_cost': 0.25, 'output_cost': 1.25}

In [22]:
def completion_cost(input_tokens: int, output_tokens: int, input_cost: float, output_cost: float):
    i_cost_tok = input_cost / 1000000
    o_cost_tok = output_cost / 1000000
    return input_tokens * i_cost_tok + output_cost * o_cost_tok

In [23]:
completion_cost(**completion_tokens, **haiku_cost)

0.0006075625

In [24]:
completion_tokens

{'input_tokens': 2424, 'output_tokens': 645}

In [18]:
.25 / 1000000

2.5e-07